In [37]:
import pandas as pd
import json

users = pd.read_csv('relax_challenge/takehome_users.csv',  encoding='cp1252', engine='python')
users.head(20)

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
5,6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
9,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0


In [10]:
users['invited_by_user_id'] = users['invited_by_user_id'].fillna(0).apply(int).apply(str).replace('0','')
users['object_id'] = users['object_id'].apply(str)
users['creation_time'] = pd.to_datetime(users['creation_time'])
users['creation_source'] = users['creation_source'].astype('category')
users['opted_in_to_mailing_list'] = users['opted_in_to_mailing_list'].astype('category')
users['enabled_for_marketing_drip'] = users['enabled_for_marketing_drip'].astype('category')
users['org_id'] = users['org_id'].astype('category')
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'],unit='s') # unix timestamp


In [11]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240


In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null object
creation_time                 12000 non-null datetime64[ns]
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null category
last_session_creation_time    8823 non-null datetime64[ns]
opted_in_to_mailing_list      12000 non-null category
enabled_for_marketing_drip    12000 non-null category
org_id                        12000 non-null category
invited_by_user_id            12000 non-null object
dtypes: category(4), datetime64[ns](2), object(4)
memory usage: 644.8+ KB


<B>Defining an "adopted user" as a user who has logged into the product on three separate days in at least one seven-day period, identify which factors predict future user adoption.

We suggest spending 1-2 hours on this, but you're welcome to spend more or less. Please send us a brief writeup of your findings (the more concise, the better -- no more than one page), along with any summary tables, graphs, code, or queries that can help us understand your approach. Please note any factors you considered or investigation you did, even if they did not pan out. Feel free to identify any further research or data you think would be valuable.</B>

In [13]:
engagement = pd.read_csv('relax_challenge/takehome_user_engagement.csv',  encoding='cp1252', engine='python')
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp'])
engagement['user_id'] = engagement['user_id'].apply(str)
engagement['visited'] = engagement['visited'].apply(str)
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [14]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null object
visited       207917 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 4.8+ MB


In [59]:
sorted =engagement.sort_values(by=['visited','time_stamp','user_id'])

In [69]:
sorted.loc[(sorted.user_id == '20' ),]

,time_stamp,user_id,visited
310,2014-03-11 11:46:38,20,1
311,2014-03-12 11:46:38,20,1
312,2014-03-13 11:46:38,20,1
313,2014-05-07 11:46:38,20,1
314,2014-05-11 11:46:38,20,1
315,2014-05-28 11:46:38,20,1
316,2014-05-29 11:46:38,20,1


In [62]:
user = sorted['user_id'].unique()

In [63]:
len(user)

8823

In [104]:
print(sorted['visited'].max(), sorted['visited'].min())

1 1


In [145]:
#counter = 0
adopted = set()
for i in user:
    visited = sorted.loc[(sorted.user_id == i),]
    #print('user '+ i  + "; total visited " + str(len(visited)))
    for j in visited['time_stamp']:
        k = j + pd.DateOffset(days=7)
        period = visited.loc[(visited.time_stamp >= j) & (visited.time_stamp <= k),]
        #print('user ' + i + '; start '+str(j) + "  " + '; end ' + str(k) + "; visited " +str(len(period)))
        #if len(period) >= 3:
        #    print('   visited ' + str(len(period)))
        unique = len(period['time_stamp'].dt.date.unique())
        if unique != len(period):
            print('same day visit for user ' +i)
        if  unique >= 3:
            adopted.add(i)
            break
            #print('   adopted ' + str(unique))
        
    #counter +=1
    #if counter > 1000:
        #break
        

In [146]:
print(len(adopted))


1656


In [149]:
print(adopted)

{'5830', '9694', '2570', '10277', '4720', '11072', '953', '11642', '8884', '725', '10459', '8201', '9965', '11957', '912', '2300', '8741', '10757', '11169', '2322', '5329', '80', '135', '4402', '874', '10233', '7380', '4009', '11519', '9400', '4565', '4218', '3765', '4388', '9269', '9968', '214', '11559', '10121', '10006', '3601', '6148', '8945', '3556', '11083', '6126', '230', '6135', '4186', '4350', '10524', '1581', '33', '5432', '5685', '3956', '11541', '2502', '8070', '8191', '10507', '9046', '2969', '8048', '8104', '10827', '11596', '11662', '3145', '10352', '9281', '4446', '11964', '5704', '3238', '3612', '6279', '10355', '160', '1993', '9290', '1506', '9728', '6285', '8442', '469', '5057', '9390', '10630', '5915', '1944', '3819', '3758', '8679', '10222', '10876', '11895', '2282', '5200', '2103', '10299', '3717', '5351', '3204', '3039', '8239', '8789', '2074', '8255', '5571', '9679', '6463', '10840', '8553', '1786', '3739', '553', '7409', '2979', '9434', '5681', '5528', '1009', '

In [148]:
len(adopted) / len(user) 

0.18769126147568854

<font color='red'>

## The above fund around 18% of users is the "adopted user". 

## The first thought was to use the rolling window. However, as the timestamp is not a daily record, the specification for the rolling window does not fit the requirement.  

## Therefore, after sorting the data according to the logged time, a dual loop was executed to find the user who is qualified.  

## The complexity is n^2, which is not the best algorithm but does perform the job.

</font>  